In [ ]:
from transformers import AutoTokenizer
from tokenization_llama_Lookup import LlamaTokenizer
import pandas as pd
import glob

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('mistralai/Mistral-7B-v0.1')

tokenizer_MEDVOC = AutoTokenizer.from_pretrained('./Mistral-7B-PubMedQA-MEDVOC/PubMedQA_Vocabs_MEDVOC_Mistral/25K_0.75_')
tokenizer_MEDVOC_ABPE = LlamaTokenizer.from_pretrained('./Mistral-7B-PubMedQA-MEDVOC/PubMedQA_Vocabs_MEDVOC_Mistral/25K_0.75_',
                                                             added_vocab = './Mistral-7B-PubMedQA-MEDVOC/PubMedQA_Vocabs_MEDVOC_Mistral/25K_0.75_/added_vocab.txt',
                                                             use_fast=False)


tokenizer_filter = AutoTokenizer.from_pretrained('./Mistral-7B-PubMedQA-Filter/PubMedQA_Filter_Vocabs_Mistral/20K')
tokenizer_filter_ABPE = LlamaTokenizer.from_pretrained('./Mistral-7B-PubMedQA-Filter/PubMedQA_Filter_Vocabs_Mistral/20K',
                                                             added_vocab = './Mistral-7B-PubMedQA-Filter/PubMedQA_Filter_Vocabs_Mistral/20K/added_vocab.txt',
                                                             use_fast=False)

tokenizer_lookup = LlamaTokenizer.from_pretrained('./Mistral-7B-PubMedQA-MedicalLookup-Fragment/PubMedQA_Lookup_SplitMoreThan1_Also_Add_As_Subwords_Mistral/PubMedQA_Lookup_250',
                                                 added_vocab = './Mistral-7B-PubMedQA-MedicalLookup-Fragment/PubMedQA_Lookup_SplitMoreThan1_Also_Add_As_Subwords_Mistral/PubMedQA_Lookup_250/added_vocab.txt',
                                                 use_fast=False)

In [ ]:
len(tokenizer), len(tokenizer_MEDVOC),len(tokenizer_MEDVOC_ABPE),len(tokenizer_filter), len(tokenizer_filter_ABPE), len(tokenizer_lookup)

In [ ]:
df_TGT  = pd.read_csv("../../LlaMa-2-7B/Llama-2-7B-PubMedQA-MedicalLookup-Fragment/PubMedQA_test.csv")
list_rs = df_TGT['target'].to_list()
list_rs = [x+'||'+str(idx) for idx,x in enumerate(list_rs)]

gs_base = open('./PubMedQA-Summaries/PubMedQA_50_4_Mistral_Base.txt','r').read().split('\n++++++++++++++++++\n')
gs_BioMistral = open('./PubMedQA-Summaries/PubMedQA_75_4_BioMistral_Base.txt','r').read().split('\n++++++++++++++++++\n')
gs_cpt_without_vocab = open('./PubMedQA-Summaries/PubMedQA_50_4_Mistral_WithoutVocab.txt','r').read().split('\n++++++++++++++++++\n')

gs_cpt_medvoc = open('./PubMedQA-Summaries/PubMedQA_40_2__Mistral_MEDVOC_WithoutAdaptBPE.txt','r').read().split('\n++++++++++++++++++\n')
gs_cpt_medvoc_adaptbpe = open('./PubMedQA-Summaries/PubMedQA_40_4__Mistral_MEDVOC_WithAdaptBPE.txt','r').read().split('\n++++++++++++++++++\n')

gs_cpt_filtered_without_adaptbpe = open('./PubMedQA-Summaries/PubMedQA_40_4__Mistral_Filter_WithoutAdaptBPE.txt','r').read().split('\n++++++++++++++++++\n')
gs_cpt_filtered_adaptbpe = open('./PubMedQA-Summaries/PubMedQA_40_2_Mistral_Filter_WithAdaptBPE.txt','r').read().split('\n++++++++++++++++++\n')

gs_cpt_lookup = open('./PubMedQA-Summaries/PubMedQA_40_2_Mistral_Lookup250.txt','r').read().split('\n++++++++++++++++++\n')

In [ ]:
from collections import defaultdict
dict_cmp = defaultdict(list)

for gs,rs in zip(gs_base, list_rs):
    dict_cmp[rs].append(gs)

for gs,rs in zip(gs_BioMistral, list_rs):
    dict_cmp[rs].append(gs)

for gs,rs in zip(gs_cpt_without_vocab, list_rs):
    dict_cmp[rs].append(gs)

for gs,rs in zip(gs_cpt_medvoc, list_rs):
    dict_cmp[rs].append(gs)

for gs,rs in zip(gs_cpt_medvoc_adaptbpe, list_rs):
    dict_cmp[rs].append(gs)

for gs,rs in zip(gs_cpt_filtered_without_adaptbpe, list_rs):
    dict_cmp[rs].append(gs)

for gs,rs in zip(gs_cpt_filtered_adaptbpe, list_rs):
    dict_cmp[rs].append(gs)

for gs,rs in zip(gs_cpt_lookup, list_rs):
    dict_cmp[rs].append(gs)


In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
def RL(ref, hyp):
    scores = scorer.score(ref,hyp)
    return scores['rougeL'].fmeasure

In [ ]:
for rs,gss in dict_cmp.items():
  try:
    rl_base = RL(rs.split('||')[0], gss[0])
    rl_biomistral = RL(rs.split('||')[0], gss[1])
    rl_cpt_without_vocab = RL(rs.split('||')[0], gss[2])
    
    rl_cpt_medvoc = RL(rs.split('||')[0], gss[3])
    rl_cpt_medvoc_adaptbpe = RL(rs.split('||')[0], gss[4])
    
    rl_cpt_filtered_without_adaptbpe = RL(rs.split('||')[0], gss[5])
    rl_cpt_filtered_adaptbpe = RL(rs.split('||')[0], gss[6])
    
    rl_cpt_lookup = RL(rs.split('||')[0], gss[7])
    
    dict_cmp[rs].extend([rl_base,rl_biomistral,rl_cpt_without_vocab,rl_cpt_medvoc,rl_cpt_medvoc_adaptbpe,rl_cpt_filtered_without_adaptbpe,rl_cpt_filtered_adaptbpe,rl_cpt_lookup])
    
  except: print(rs)



In [ ]:
list_src = df_TGT['inputs'].to_list()


In [ ]:
list_src[0], gs_base[0], list_rs[0]

In [ ]:
import re
pattern = r"^[A-Za-z▁]+$"

def OOV_frac(tokenizer,rs):
    tokens = rs.split()
    
    oov_frac = 0
    consider = 0
    oov_list = list()
    for token in tokens:
        if re.match(pattern,token): 
            consider += 1
            if len(tokenizer.tokenize(token)) > 1: 
                oov_frac += 1
                
    return oov_frac/consider

def Novel_frac(src,rs):
    novel_frac = 0
    novel_list = list()
    for tok in rs.split():
        if tok not in src: 
            novel_frac += 1
            novel_list.append(tok)
    
    return novel_frac/len(rs.split()), novel_list

In [ ]:
OOV_frac(tokenizer,list_rs[10]), list_rs[10]

In [ ]:
dict_src = defaultdict(list)
for rs,src in zip(list_rs,list_src): dict_src[rs].append(src)

list_del_key = list()
for key,val in dict_src.items():
    # print(key,val)
    if len(val[0]) == 0: 
        list_del_key.append(key)
        continue
        
    dict_src[key].extend(Novel_frac(val[0].lower(),key))
    dict_src[key].append(OOV_frac(tokenizer,val[0]))
    dict_src[key].append(OOV_frac(tokenizer_MEDVOC,val[0]))
    dict_src[key].append(OOV_frac(tokenizer_MEDVOC_ABPE,val[0]))
    dict_src[key].append(OOV_frac(tokenizer_filter,val[0]))
    dict_src[key].append(OOV_frac(tokenizer_filter_ABPE,val[0]))
    dict_src[key].append(OOV_frac(tokenizer_lookup,val[0]))

In [ ]:
list_rl_base = list()
list_rl_biomistral = list()
list_rl_cpt_without_vocab = list()
list_rl_cpt_medvoc = list()
list_rl_cpt_medvoc_adaptbpe = list()
list_rl_cpt_filtered_without_adaptbpe = list()
list_rl_cpt_filtered_adaptbpe = list()
list_rl_cpt_lookup = list()

list_oov_src_base = list()
list_oov_src_medvoc = list()
list_oov_src_medvoc_adaptbpe = list()
list_oov_src_filtered_without_adaptbpe = list()
list_oov_src_filtered_adaptbpe = list()
list_oov_src_lookup = list()

list_oov_rs_base = list()
list_oov_rs_medvoc = list()
list_oov_rs_medvoc_adaptbpe = list()
list_oov_rs_filtered_without_adaptbpe = list()
list_oov_rs_filtered_adaptbpe = list()
list_oov_rs_lookup = list()

list_len_rs_base = list()
list_len_rs_medvoc = list()
list_len_rs_medvoc_adaptbpe = list()
list_len_rs_filtered_without_adaptbpe = list()
list_len_rs_filtered_adaptbpe = list()
list_len_rs_lookup = list()

list_novel_rs = list()

list_gs_base = list()
list_gs_biomistral = list()
list_gs_cpt_without_vocab = list()
list_gs_cpt_medvoc = list()
list_gs_cpt_medvoc_adaptbpe = list()
list_gs_cpt_filtered_without_adaptbpe = list()
list_gs_cpt_filtered_adaptbpe = list()
list_gs_cpt_lookup = list()

list_rs = list()
list_sd = list()

for rs in dict_cmp:
  try:
    gen_base,gen_biomistral,gen_cpt_without_vocab,gen_medvoc,gen_medvoc_adaptbpe,gen_cpt_filtered_without_adaptbpe,gen_cpt_filtered_adaptbpe,gen_cpt_lookup,rl_base,rl_biomistral,rl_cpt_without_vocab,rl_cpt_medvoc,rl_cpt_medvoc_adaptbpe,rl_cpt_filtered_without_adaptbpe,rl_cpt_filtered_adaptbpe,rl_cpt_lookup = dict_cmp[rs]
    
    sd = dict_src[rs][0]
    
    len_rs_base = len(tokenizer.tokenize(rs))
    len_rs_medvoc = len(tokenizer_MEDVOC.tokenize(rs))
    len_rs_medvoc_adaptbpe = len(tokenizer_MEDVOC_ABPE.tokenize(rs))
    len_rs_filtered_without_adaptbpe = len(tokenizer_filter.tokenize(rs))
    len_rs_filtered_adaptbpe = len(tokenizer_filter_ABPE.tokenize(rs))
    len_rs_lookup = len(tokenizer_lookup.tokenize(rs))

    oov_sd_base = dict_src[rs][-6]
    oov_sd_medvoc = dict_src[rs][-5]
    oov_sd_medvoc_adaptbpe = dict_src[rs][-4]
    oov_sd_filtered_without_adaptbpe = dict_src[rs][-3]
    oov_sd_filtered_adaptbpe = dict_src[rs][-2]
    oov_sd_lookup= dict_src[rs][-1]

    oov_rs_base = OOV_frac(tokenizer,rs)
    oov_rs_medvoc = OOV_frac(tokenizer_MEDVOC,rs)
    oov_rs_medvoc_adaptbpe = OOV_frac(tokenizer_MEDVOC_ABPE,rs)
    oov_rs_filtered_without_adaptbpe = OOV_frac(tokenizer_filter,rs)
    oov_rs_filtered_adaptbpe = OOV_frac(tokenizer_filter_ABPE,rs)
    oov_rs_lookup = OOV_frac(tokenizer_lookup,rs)
    
    novel_rs = dict_src[rs][1]
    
    list_rl_base.append(rl_base)
    list_rl_biomistral.append(rl_biomistral)
    list_rl_cpt_without_vocab.append(rl_cpt_without_vocab)
    list_rl_cpt_medvoc.append(rl_cpt_medvoc)
    list_rl_cpt_medvoc_adaptbpe.append(rl_cpt_medvoc_adaptbpe)
    list_rl_cpt_filtered_without_adaptbpe.append(rl_cpt_filtered_without_adaptbpe)
    list_rl_cpt_filtered_adaptbpe.append(rl_cpt_filtered_adaptbpe)
    list_rl_cpt_lookup.append(rl_cpt_lookup)
    
    list_len_rs_base.append(len_rs_base)
    list_len_rs_medvoc.append(len_rs_medvoc)
    list_len_rs_medvoc_adaptbpe.append(len_rs_medvoc_adaptbpe)
    list_len_rs_filtered_without_adaptbpe.append(len_rs_filtered_without_adaptbpe)
    list_len_rs_filtered_adaptbpe.append(len_rs_filtered_adaptbpe)
    list_len_rs_lookup.append(len_rs_lookup)
    
    list_gs_base.append(gen_base)
    list_gs_biomistral.append(gen_biomistral)
    list_gs_cpt_medvoc.append(gen_medvoc)
    list_gs_cpt_medvoc_adaptbpe.append(gen_medvoc_adaptbpe)
    list_gs_cpt_without_vocab.append(gen_cpt_without_vocab)
    list_gs_cpt_filtered_without_adaptbpe.append(gen_cpt_filtered_without_adaptbpe)
    list_gs_cpt_filtered_adaptbpe.append(gen_cpt_filtered_adaptbpe)
    list_gs_cpt_lookup.append(gen_cpt_lookup)
    
    list_rs.append(rs)
    list_sd.append(sd)
    
    list_novel_rs.append(novel_rs)

    list_oov_rs_base.append(oov_rs_base)
    list_oov_rs_medvoc.append(list_oov_rs_medvoc)
    list_oov_rs_medvoc_adaptbpe.append(list_oov_rs_medvoc_adaptbpe)
    list_oov_rs_filtered_without_adaptbpe.append(oov_rs_filtered_without_adaptbpe)
    list_oov_rs_filtered_adaptbpe.append(oov_rs_filtered_adaptbpe)
    list_oov_rs_lookup.append(oov_rs_lookup)

    list_oov_src_base.append(oov_sd_base)
    list_oov_src_medvoc.append(oov_sd_medvoc)
    list_oov_src_medvoc_adaptbpe.append(oov_sd_medvoc_adaptbpe)
    list_oov_src_filtered_without_adaptbpe.append(oov_sd_filtered_without_adaptbpe)
    list_oov_src_filtered_adaptbpe.append(oov_sd_filtered_adaptbpe)
    list_oov_src_lookup.append(oov_sd_lookup)

  except Exception as e:
    print(e)
#     print(rs)
    print(len(dict_cmp[rs]), len(dict_src[rs]))
    pass

In [ ]:
import pandas as pd

df = pd.DataFrame({'SD': list_sd,'RS':list_rs, \
                   'GS_Base': list_gs_base , 'GS_BioMistral': list_gs_biomistral,'GS_CPT_Without_Vocab': list_gs_cpt_without_vocab, \
                   'GS_CPT_MedVoc': list_gs_cpt_medvoc,'GS_CPT_MedVoc_AdaptBPE': list_gs_cpt_medvoc_adaptbpe, \
                   'GS_CPT_Filtering_WithoutAdaptBPE': list_gs_cpt_filtered_without_adaptbpe, \
                   'GS_CPT_Filtering_AdaptBPE': list_gs_cpt_filtered_adaptbpe, \
                   'GS_CPT_Lookup':list_gs_cpt_lookup, \
                   'RS_Len_Base': list_len_rs_base, 'RS_Len_Filtering_WithoutAdaptBPE': list_len_rs_filtered_without_adaptbpe, \
                   'RS_Len_MedVoc': list_len_rs_medvoc, 'RS_Len_MedVoc_AdaptBPE': list_len_rs_medvoc_adaptbpe, \
                   'RS_Len_Filtering_AdaptBPE': list_len_rs_filtered_adaptbpe, \
                   'RS_Len_Lookup' : list_len_rs_lookup, \
                   'OOV_RS_Base': list_oov_rs_base, 'OOV_RS_Filtering_WithoutAdaptBPE': list_oov_rs_filtered_without_adaptbpe, \
                   'OOV_RS_MedVoc': list_oov_rs_medvoc, 'OOV_RS_MedVoc_AdaptBPE': list_oov_rs_medvoc_adaptbpe, \
                   'OOV_RS_Filtering_AdaptBPE': list_oov_rs_filtered_adaptbpe, \
                   'OOV_RS_Lookup' : list_oov_rs_lookup, \
                   'Novel_RS': list_novel_rs, \
                   'OOV_SD_Base': list_oov_src_base, 'OOV_SD_Filtering_WithoutAdaptBPE': list_oov_src_filtered_without_adaptbpe, \
                   'OOV_SD_MedVoc': list_oov_src_medvoc, 'OOV_SD_MedVoc_AdaptBPE': list_oov_src_medvoc_adaptbpe, \
                   'OOV_SD_Filtering_AdaptBPE': list_oov_src_filtered_adaptbpe, \
                   'OOV_SD_Lookup' : list_oov_src_lookup, \
                   'R-L_Base': list_rl_base , 'R-L_BioMistral':list_rl_biomistral, 'R-L_CPT_Without_Vocab': list_rl_cpt_without_vocab, \
                   'R-L_CPT_MedVoc': list_rl_cpt_medvoc, 'R-L_CPT_MedVoc_AdaptBPE': list_rl_cpt_medvoc_adaptbpe, \
                   'R-L_CPT_Filtering_WithoutAdaptBPE': list_rl_cpt_filtered_without_adaptbpe, \
                   'R-L_CPT_Filtering_AdaptBPE': list_rl_cpt_filtered_adaptbpe, \
                   'R-L_CPT_Lookup':list_rl_cpt_lookup
                   })



In [ ]:
df.to_csv('./PubMedQA-Summaries/PubMedQA_Mistral_Compare_WithBioMistral.csv',index=False)

In [ ]:
import pandas as pd
df = pd.read_csv('./PubMedQA-Summaries/PubMedQA_Mistral_Compare.csv')

In [ ]:
import argparse
from quickumls import QuickUMLS
import pandas as pd
import glob


umls_path = '/path/to/umls'
matcher = QuickUMLS(umls_path,similarity_name='cosine',threshold=0.95) 

In [ ]:
from collections import defaultdict
import json
counter_TGT_RS = defaultdict(list)

lines_TGT = df['SD'].to_list()

for idx,abs in enumerate(lines_TGT):
    if idx%10 == 0: print(f'Processed till {idx+1}... ')

    flag = 0
    d = matcher.match(abs, best_match=True, ignore_syntax=False)
    if len(d) == 0:
        print('No match found for',idx,abs)
        # counter_TGT_RS[idx].append('happy')
    
    else:
        for l in d:
            counter_TGT_RS[idx].append(l[0]['ngram'])

import re
pattern = r"^[A-Za-z▁]+$"

counter_TGT_RS_Medical_Words = defaultdict(set)

for key,val in counter_TGT_RS.items():
    try:
      print('---------')
      for words in val:
        for word in words.split():
          print(word)
          if re.match(pattern,word.strip()): counter_TGT_RS_Medical_Words[key].add(word.strip())
    except: print("Error in", key,val)

for key,val in counter_TGT_RS_Medical_Words.items():
    print(key,val)
    print('-----------------')

import pickle as pkl
with open('./PubMedQA-Summaries/PubMedQA_Mistral_MedicalWords_SD.pkl','wb') as f:
    pkl.dump(counter_TGT_RS_Medical_Words,f)
f.close()


In [ ]:
len(counter_TGT_RS_Medical_Words)

In [ ]:
import pickle as pkl
import pandas as pd
# df = pd.read_csv('./PubMedQA-Summaries/PubMedQA_Mistral_Compare_MedicalWords.csv')

with open('./PubMedQA-Summaries/PubMedQA_Mistral_MedicalWords_SD.pkl','rb') as f:
    counter_TGT_RS_Medical_Words = pkl.load(f)
f.close()

In [ ]:
list_medical_words_splitmorethan1 = list()
list_medical_words_splitmorethan2 = list()
list_medical_words_splitmorethan3 = list()


for idx, words in counter_TGT_RS_Medical_Words.items():
    total_words = len(words)
    splitmorethan1 = 0
    splitmorethan2 = 0
    splitmorethan3 = 0
    
    for word in words:
        if len(tokenizer.tokenize(word)) > 1: splitmorethan1 += 1
        if len(tokenizer.tokenize(word)) > 2: splitmorethan2 += 1
        if len(tokenizer.tokenize(word)) > 3: splitmorethan3 += 1
    
    list_medical_words_splitmorethan1.append(splitmorethan1/total_words)
    list_medical_words_splitmorethan2.append(splitmorethan2/total_words)
    list_medical_words_splitmorethan3.append(splitmorethan3/total_words)


In [ ]:
df['Medical_Words_SplitMoreThan1_SD'] = list_medical_words_splitmorethan1
df['Medical_Words_SplitMoreThan2_SD'] = list_medical_words_splitmorethan2
df['Medical_Words_SplitMoreThan3_SD'] = list_medical_words_splitmorethan3

df.to_csv('./PubMedQA-Summaries/PubMedQA_Mistral_Compare_MedicalWords_WithBioMistral.csv',index=False)

In [4]:
import pandas as pd
df = pd.read_csv('./PubMedQA-Summaries/PubMedQA_Mistral_Compare_MedicalWords_WithBioMistral.csv')
df['Medical_Words_SplitMoreThan1_TGT'].describe(percentiles=[0.1,0.9])

count    500.000000
mean       0.277306
std        0.173002
min        0.000000
10%        0.066667
50%        0.266667
90%        0.500000
max        1.000000
Name: Medical_Words_SplitMoreThan1_TGT, dtype: float64

In [6]:
df_high_oov = df[df['Medical_Words_SplitMoreThan1_TGT'] >= 0.500000]
df_desc = pd.DataFrame()
for col in df_high_oov.columns:
      if col.startswith('CSr') or col.startswith('R-L') : df_desc[col] = df_high_oov[col]
df_desc.describe(percentiles=[0.5])

,R-L_Base,R-L_BioMistral,R-L_CPT_Without_Vocab,R-L_CPT_MedVoc,R-L_CPT_MedVoc_AdaptBPE,R-L_CPT_Filtering_WithoutAdaptBPE,R-L_CPT_Filtering_AdaptBPE,R-L_CPT_Lookup,CSr_Base,CSr_BioMistral,CSr_CPT_Without_Vocab,CSr_CPT_MedVoc,CSr_CPT_MedVoc_AdaptBPE,CSr_CPT_Filtering_WithoutAdaptBPE,CSr_CPT_Filtering_AdaptBPE,CSr_CPT_Lookup
count,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000
mean,0.304875,0.319580,0.297978,0.277136,0.269739,0.290573,0.296606,0.312748,0.264622,0.320408,0.308327,0.281449,0.300740,0.314065,0.326538,0.320826
std,0.117990,0.151156,0.122261,0.104065,0.107241,0.094404,0.112241,0.128621,0.150485,0.205796,0.206871,0.187670,0.190157,0.191541,0.211858,0.217523
min,0.120000,0.061538,0.037037,0.072727,0.045455,0.045455,0.045455,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.268468,0.298936,0.276144,0.253571,0.253343,0.285714,0.272727,0.280625,0.250000,0.292857,0.314145,0.273292,0.296703,0.324561,0.307692,0.319185
max,0.590909,0.714286,0.600000,0.533333,0.490566,0.533333,0.528302,0.600000,0.666667,0.857143,0.823529,0.750000,0.800000,0.666667,0.857143,0.750000


High Expert OOV Concentration (Terms Split more than once)
Base        BioMistral  W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.268468	0.298936	0.276144	0.253571	0.253343	0.285714	0.272727	0.280625

High Expert OOV Concentration (Terms Split more than thrice)
Base        BioMistral  W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.257143	0.250000	0.241758	0.254545	0.253165	0.245614	0.244898	0.274510	

High Expert OOV Concentration -SD (Terms split more than once)
Base        BioMistral  W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.211324	0.243163	0.252273	0.245482	0.230696	0.241902	0.242424	0.252273

High Expert OOV Concentration -SD (Terms split more than thrice)
Base        BioMistral  W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.266088	0.250000	0.259006	0.245482	0.256902	0.251582	0.264151	0.273619

High Novelty
Base        BioMistral  W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.182105	0.196687    0.188988	0.193548	0.200000	0.205345	0.205263	0.199716

Low Novelty
Base        BioMistral  W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.342857	0.380952    0.352941	0.338028	0.275862	0.346154	0.322581	0.380952

Long RS (All)
Base        W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.209302	0.222222	0.217822	0.211268	0.222222	0.218182	0.222222

Low Expert OOV Concentration (Terms Split more than once)
Base        W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup	
0.250000	0.225806	0.255319	0.243902	0.264151	0.250000	0.257143

Low Expert OOV Concentration (Terms Split more than thrice)
Base        W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.240000	0.250000	0.246575	0.243902	0.258065	0.245614	0.259740

Low Expert OOV Concentration -SD (Terms split more than once)
Base        W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.210526	0.213592	0.208955	0.225000	0.215385	0.217822	0.227848

Low Expert OOV Concentration -SD (Terms split more than thrice)
Base        W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.236754	0.232051	0.237683	0.235065	0.247191	0.236842	0.251761


In [ ]:
df_long_rs = df[df['RS_Len_Base'] >= 92]
df_long_rs.describe(percentiles=[0.5])

In [ ]:
df_RS_hHighOOV = df_high_oov[df_high_oov['RS_Len_Base']>=89]
df_RS_hHighOOV.describe(percentiles=[0.5])

In [ ]:
from collections import defaultdict

def getMatchInfo(list_matches):
    dict_concepts = defaultdict(list)
    for match_l in list_matches:
        for match_l_d in match_l:
            if match_l_d["preferred"] == 1: 
                cui = match_l_d['cui'] #the concept-id
                start = match_l_d['start'] 
                end = match_l_d['end']
                n_gram = match_l_d['ngram'].strip() #the surface form

                if '\n' in n_gram : continue
                key = str(start)+'_'+str(end)+'_'+n_gram

                if not cui in dict_concepts[key]: dict_concepts[key].append(cui)
                    
    str_ret = str()
    
    for key,val in dict_concepts.items():
        str_ret += str(key) + ':'
        for c_name in val:
            if not '\n' in c_name:
                str_ret += c_name + '|'
        str_ret += '\n'
    return (str_ret)


import numpy as np
import glob
import os
import pandas as pd

def check_overlap(concepts_ref, concepts_dec,verbose=False):
    list_concepts_ref = concepts_ref.splitlines()
    list_concepts_dec = concepts_dec.splitlines()
    
    cuis_dec, cuis_ref = list(), list()
    
    for line in list_concepts_ref:
        cui_ref = line.split(':')[1].split('|')[:-1]
        cuis_ref.extend(cui_ref)
    
    for line in list_concepts_dec:
        cui_dec = line.split(':')[1].split('|')[:-1]
        cuis_dec.extend(cui_dec)
    
    cuis_ref = set(cuis_ref)
    cuis_dec = set(cuis_dec)
    
    
    common_cuis = cuis_dec.intersection(cuis_ref)
    
    if verbose: print('Common_cuis:', common_cuis)
    
    if len(common_cuis) == 0: return 0.
    
    prec = len(common_cuis)/len(cuis_dec)
    rec = len(common_cuis)/len(cuis_ref)
    
    return ((2*prec*rec)/(prec+rec))
import numpy as np
import glob
import os
import pandas as pd

def check_overlap(concepts_ref, concepts_dec,verbose=False):
    list_concepts_ref = concepts_ref.splitlines()
    list_concepts_dec = concepts_dec.splitlines()
    
    cuis_dec, cuis_ref = list(), list()
    
    for line in list_concepts_ref:
        cui_ref = line.split(':')[1].split('|')[:-1]
        cuis_ref.extend(cui_ref)
    
    for line in list_concepts_dec:
        cui_dec = line.split(':')[1].split('|')[:-1]
        cuis_dec.extend(cui_dec)
    
    cuis_ref = set(cuis_ref)
    cuis_dec = set(cuis_dec)
    
    
    common_cuis = cuis_dec.intersection(cuis_ref)
    
    if verbose: print('Common_cuis:', common_cuis)
    
    if len(common_cuis) == 0: return 0.
    
    prec = len(common_cuis)/len(cuis_dec)
    rec = len(common_cuis)/len(cuis_ref)
    
    return ((2*prec*rec)/(prec+rec))
from numpy.random import seed
from numpy.random import rand
from numpy.random import randint
from numpy import mean
from numpy import median
from numpy import percentile
import numpy as np
def CIEval(f_score):
    dataset = np.array([x[1] for x in f_score])
    max_l = len(dataset)
    scores = list()
    for _ in range(1000):
        # bootstrap sample
        indices = randint(0, max_l, max_l)
        sample = dataset[indices]
        # calculate and store statistic
        statistic = mean(sample)
        scores.append(statistic)

    print('50th percentile (median) = %.4f' % median(scores))
    # calculate 95% confidence intervals (100 - alpha)
    alpha = 5.0
    # calculate lower percentile (e.g. 2.5)
    lower_p = alpha / 2.0
    # retrieve observation at lower percentile
    lower = max(0.0, percentile(scores, lower_p))
#     print('%.1fth percentile = %.4f' % (lower_p, lower))
    # calculate upper percentile (e.g. 97.5)
    upper_p = (100 - alpha) + (alpha / 2.0)
    # retrieve observation at upper percentile
    upper = min(1.0, percentile(scores, upper_p))
#     print('%.1fth percentile = %.4f' % (upper_p, upper))
    print('C.I. Window = %.4f '%max([upper-median(scores),median(scores)-lower]))
for column in ['GS_Base', 'GS_BioMistral', 'GS_CPT_Without_Vocab', 'GS_CPT_MedVoc',
       'GS_CPT_MedVoc_AdaptBPE', 'GS_CPT_Filtering_WithoutAdaptBPE',
       'GS_CPT_Filtering_AdaptBPE', 'GS_CPT_Lookup']:

    print(f'--Processing {column}.....')

    f_score = list()
    for row_id in range(df.shape[0]):
        try:                
            dec_sum = df.iloc[row_id][column]
            ref_sum = df.iloc[row_id]['RS']
            ref_sum = ref_sum.split('||')[0]
            
            ref_con = getMatchInfo(matcher.match(ref_sum, best_match=True, ignore_syntax=False))
            dec_con = getMatchInfo(matcher.match(dec_sum, best_match=True, ignore_syntax=False))
            
            # print(f'REF_Sum: {ref_sum}')
            # print(f'DEC_Sum: {dec_sum}')

            # print(check_overlap(ref_con,dec_con))
            f_score.append((row_id,check_overlap(ref_con,dec_con)))
            
        except Exception as e: 
            print(e)
            
#         break
#     break
    df['CSr'+column[2:]] = [x[1] for x in f_score]
    CIEval(f_score)



In [ ]:
df.to_csv('./PubMedQA-Summaries/PubMedQA_Mistral_Compare_MedicalWords_WithBioMistral.csv',index=False)

In [ ]:
import pandas as pd


In [ ]:
df_high_oov = df[df['Medical_Words_SplitMoreThan1_TGT'] >= 0.500000]
df_desc = df_high_oov.drop(columns=['GS_CPT_MedVoc',
       'GS_CPT_MedVoc_AdaptBPE', 'GS_CPT_Filtering_WithoutAdaptBPE',
       'GS_CPT_Filtering_AdaptBPE', 'RS_Len_Base',
       'RS_Len_Filtering_WithoutAdaptBPE', 'RS_Len_MedVoc',
       'RS_Len_MedVoc_AdaptBPE', 'RS_Len_Filtering_AdaptBPE', 'RS_Len_Lookup',
       'OOV_RS_Base', 'OOV_RS_Filtering_WithoutAdaptBPE', 'OOV_RS_MedVoc',
       'OOV_RS_MedVoc_AdaptBPE', 'OOV_RS_Filtering_AdaptBPE', 'OOV_RS_Lookup',
       'Novel_RS', 'OOV_SD_Base', 'OOV_SD_Filtering_WithoutAdaptBPE',
       'OOV_SD_MedVoc', 'OOV_SD_MedVoc_AdaptBPE', 'OOV_SD_Filtering_AdaptBPE',
       'OOV_SD_Lookup'])
df_desc.describe(percentiles=[0.5])
df_desc = df_desc[df_desc['R-L_Base']<df_desc['R-L_CPT_Lookup']]
df_desc = df_desc[df_desc['R-L_CPT_Without_Vocab']<df_desc['R-L_CPT_Lookup']]
df_desc_greatest = df_desc.nlargest(int(20), 'R-L_CPT_Lookup')
for idx,row in df_desc_greatest.iterrows():
    print('------------------------'*8)
    print('+++SD',row['SD'])
    print('+++RS        :',row['RS'])
    print('######################################')
    print('**GS_Base    :',row['GS_Base'])
    print('**GS_W/oVocab:',row['GS_CPT_Without_Vocab'])
    print('**GS_Lookup  :',row['GS_CPT_Lookup'])

In [ ]:
from scipy.stats import wilcoxon
print(wilcoxon(df_high_oov['R-L_CPT_Without_Vocab'].to_list(),df_high_oov['R-L_CPT_Lookup'].to_list(),alternative='less'), \
wilcoxon(df_low_oov['R-L_CPT_Without_Vocab'].to_list(),df_low_oov['R-L_CPT_Lookup'].to_list(),alternative='less'))

In [ ]:
import re
pattern = r"^[A-Za-z▁]+$"

def OOV_frac(tokenizer,rs):
    tokens = rs.split()
    
    oov_frac = 0
    consider = 0
    oov_list = list()
    for token in tokens:
        if re.match(pattern,token): 
            consider += 1
            if len(tokenizer.tokenize(token)) > 1: 
                oov_frac += 1
                oov_list.append([token,tokenizer.tokenize(token)])
    return oov_frac/consider, oov_list

In [ ]:
list_common_words = []
df_high_oov_better = df_high_oov[df_high_oov['R-L_CPT_Filtering_WithoutAdaptBPE']>df_high_oov['R-L_CPT_Without_Vocab']]
print(df_high_oov_better.shape, df_high_oov.shape)
df_high_oov_better = df_high_oov_better.sort_values(by='R-L_CPT_Filtering_WithoutAdaptBPE',ascending=False)
for idx,row in df_high_oov_better.iterrows():
    rs = row['RS']
    print('++++++++++++', row['R-L_CPT_Filtering_WithoutAdaptBPE'], row['R-L_CPT_Without_Vocab'],row['Novel_RS'],row['OOV_RS_Base'],row['OOV_RS_Filtering_WithoutAdaptBPE'],row['RS_Len_Base'],row['RS_Len_Filtering_WithoutAdaptBPE'])
    print('-- SD: ',row['SD'])
    print('-- RS                               :',row['RS'])
    print('***')
    print('-- GS_CPT_Without_Vocab             :',row['GS_CPT_Without_Vocab'])
    print('-- GS_CPT_Without_Filtering_AdaptBPE:',row['GS_CPT_Filtering_WithoutAdaptBPE'])
    print('-- OOV Base Tokenizer               :', OOV_frac(tokenizer,rs)) 
    print('-- Lookup-1000                      :',OOV_frac(tokenizer_lookup,rs))
    common_words = 0
    for token1 in  row['GS_CPT_Filtering_WithoutAdaptBPE'].split():
        if token1 in row['GS_CPT_Without_Vocab']: common_words+=1
        
    common_words /= len(row['GS_CPT_Filtering_WithoutAdaptBPE'].split()) 
    
    list_common_words.append(common_words)
    print('-- Common Words Frac               :', common_words)
    

In [ ]:
df_high_oov_worst = df_high_oov[df_high_oov['R-L_CPT_Filtering_WithoutAdaptBPE']<=df_high_oov['R-L_CPT_Without_Vocab']]
list_common_words_worst = []
print(df_high_oov_worst.shape, df_high_oov.shape)
df_high_oov_worst = df_high_oov_worst.sort_values(by='R-L_CPT_Filtering_WithoutAdaptBPE')
for idx,row in df_high_oov_worst.iterrows():
    rs = row['RS']
    print('++++++++++++', row['R-L_CPT_Filtering_WithoutAdaptBPE'], row['R-L_CPT_Without_Vocab'],row['Novel_RS'],row['OOV_RS_Base'],row['OOV_RS_Filtering_WithoutAdaptBPE'],row['RS_Len_Base'],row['RS_Len_Filtering_WithoutAdaptBPE'])
    print('-- SD: ',row['SD'])
    print('-- RS                               : ',row['RS'])
    print('***')
    print('-- GS_CPT_Without_Vocab             :',row['GS_CPT_Without_Vocab'])
    print('-- GS_CPT_Without_Filtering_AdaptBPE:',row['GS_CPT_Filtering_WithoutAdaptBPE'])
    print('-- OOV Base Tokenizer               :', OOV_frac(tokenizer,rs)) 
    print('-- Filtering w/o AdaptBPE           :',OOV_frac(tokenizer_filtering,rs))
    common_words = 0
    for token1 in  row['GS_CPT_Filtering_WithoutAdaptBPE'].split():
        if token1 in row['GS_CPT_Without_Vocab']: common_words+=1
        
    common_words /= len(row['GS_CPT_Filtering_WithoutAdaptBPE'].split()) 
    
    list_common_words_worst.append(common_words)
    print('-- Common words Frac                :', common_words)

In [ ]:
df.describe(percentiles=[0.1,0.25,0.5,0.75,0.9])

In [ ]:
df_high_oov = df[df['OOV_RS_Lookup']>=0.27]
df_high_oov.describe()

df_long_rs = df[df['RS_Len_Lookup']>=93]
df_long_rs.describe()

In [ ]:
df_high_oov.describe()

In [ ]:
Long-RS:
Base > CPT_Filter_W/OABPE > CPT_W/O_Vocab > CPT_Lookup > CPT_Filter_AdaptBPE
0.219003 > 0.213338 > 0.208825 > 0.205657 > 0.185030

High-OOV:
CPT_Lookup > Base > CPT_Filter_AdaptBPE > CPT_Filter_W/OABPE > CPT_W/O_Vocab
0.263158 > 0.258094 > 0.255865 > 0.251984 > 0.245526

In [ ]:
df_high_oov = df[df['OOV_RS_Filtering_WithoutAdaptBPE']>=0.25]
df_high_oov.describe()

High-OOV:
Lookup25.59 > W/O_Vocab25.00 > Base24.83 > Filter_W/OABPE22.04 > Filter_ABPE20.97


In [ ]:
df_long_rs = df[df['RS_Len_Filtering_WithoutAdaptBPE']>=89]
df_long_rs.describe()

Long-RS:
Base20.64 > Lookup20.15 > W/O_Vocab19.33 > Filter_W/OABPE18.90 > Filter_ABPE18.27

In [ ]:
list_common_words = []
df_high_oov_better = df_high_oov[df_high_oov['R-L_CPT_Lookup']>df_high_oov['R-L_CPT_Without_Vocab']]
print(df_high_oov_better.shape, df_high_oov.shape)
df_high_oov_better = df_high_oov_better.sort_values(by='R-L_CPT_Lookup',ascending=False)
for idx,row in df_high_oov_better.iterrows():
    rs = row['RS']
    print('++++++++++++', row['R-L_CPT_Lookup'], row['R-L_CPT_Without_Vocab'],row['Novel_RS'],row['OOV_RS_Base'],row['OOV_RS_Lookup'],row['RS_Len_Base'],row['RS_Len_Lookup'])
    print('-- SD: ',row['SD'])
    print('-- RS                               :',row['RS'])
    print('***')
    print('-- GS_CPT_Without_Vocab             :',row['GS_CPT_Without_Vocab'])
    print('-- GS_CPT_Without_Filtering_AdaptBPE:',row['GS_CPT_Lookup'])
    print('-- OOV Base Tokenizer               :', OOV_frac(tokenizer,rs)) 
    print('-- Lookup-1000                      :',OOV_frac(tokenizer_lookup,rs))
    common_words = 0
    for token1 in  row['GS_CPT_Lookup'].split():
        if token1 in row['GS_CPT_Without_Vocab']: common_words+=1
        
    common_words /= len(row['GS_CPT_Lookup'].split()) 
    
    list_common_words.append(common_words)
    print('-- Common Words Frac               :', common_words)
    

In [ ]:
list_common_words = []
df_high_oov_worst = df_high_oov[df_high_oov['R-L_CPT_Lookup']<=df_high_oov['R-L_CPT_Without_Vocab']]
print(df_high_oov_worst.shape, df_high_oov.shape)
df_high_oov_better = df_high_oov_worst.sort_values(by='R-L_CPT_Lookup',ascending=False)
for idx,row in df_high_oov_worst.iterrows():
    rs = row['RS']
    print('++++++++++++', row['R-L_CPT_Lookup'], row['R-L_CPT_Without_Vocab'],row['Novel_RS'],row['OOV_RS_Base'],row['OOV_RS_Lookup'],row['RS_Len_Base'],row['RS_Len_Lookup'])
    print('-- SD: ',row['SD'])
    print('-- RS                               :',row['RS'])
    print('***')
    print('-- GS_CPT_Without_Vocab             :',row['GS_CPT_Without_Vocab'])
    print('-- GS_CPT_Without_Filtering_AdaptBPE:',row['GS_CPT_Lookup'])
    print('-- OOV Base Tokenizer               :', OOV_frac(tokenizer,rs)) 
    print('-- Lookup-1000                      :',OOV_frac(tokenizer_lookup,rs))
    common_words = 0
    for token1 in  row['GS_CPT_Lookup'].split():
        if token1 in row['GS_CPT_Without_Vocab']: common_words+=1
        
    common_words /= len(row['GS_CPT_Lookup'].split()) 
    
    list_common_words.append(common_words)
    print('-- Common Words Frac               :', common_words)
    

In [1]:
import pandas as pd
df_mistral = pd.read_csv('./PubMedQA-Summaries/PubMedQA_Mistral_Compare_MedicalWords_WithBioMistral.csv')
df_llama = pd.read_csv('../Llama-2-7b/PubMedQA-Summaries/PubMedQA_Llama2_Compare_SelfAdaptBPE_MedicalWords_WithLlama3.csv')

In [2]:
df_llama.columns, df_mistral.columns

(Index(['SD', 'RS', 'GS_Base', 'GS_Llama3.1', 'GS_CPT_Without_Vocab',
        'GS_CPT_MedVoc', 'GS_CPT_MedVoc_AdaptBPE',
        'GS_CPT_Filtering_WithoutAdaptBPE', 'GS_CPT_Filtering_AdaptBPE',
        'GS_CPT_Lookup', 'RS_Len_Base', 'RS_Len_Filtering_WithoutAdaptBPE',
        'RS_Len_MedVoc', 'RS_Len_MedVoc_AdaptBPE', 'RS_Len_Filtering_AdaptBPE',
        'RS_Len_Lookup', 'OOV_RS_Base', 'OOV_RS_Filtering_WithoutAdaptBPE',
        'OOV_RS_MedVoc', 'OOV_RS_MedVoc_AdaptBPE', 'OOV_RS_Filtering_AdaptBPE',
        'OOV_RS_Lookup', 'Novel_RS', 'OOV_SD_Base',
        'OOV_SD_Filtering_WithoutAdaptBPE', 'OOV_SD_MedVoc',
        'OOV_SD_MedVoc_AdaptBPE', 'OOV_SD_Filtering_AdaptBPE', 'OOV_SD_Lookup',
        'R-L_Base', 'R-L_Llama3.1', 'R-L_CPT_Without_Vocab', 'R-L_CPT_MedVoc',
        'R-L_CPT_MedVoc_AdaptBPE', 'R-L_CPT_Filtering_WithoutAdaptBPE',
        'R-L_CPT_Filtering_AdaptBPE', 'R-L_CPT_Lookup',
        'Medical_Words_SplitMoreThan1', 'Medical_Words_SplitMoreThan2',
        'Medical_Words

In [3]:
df_merged = pd.merge(df_mistral,df_llama,on='SD',suffixes=('_Mistral','_Llama'))

In [4]:
df_merged.columns

Index(['SD', 'RS_Mistral', 'GS_Base_Mistral', 'GS_BioMistral',
       'GS_CPT_Without_Vocab_Mistral', 'GS_CPT_MedVoc_Mistral',
       'GS_CPT_MedVoc_AdaptBPE_Mistral',
       'GS_CPT_Filtering_WithoutAdaptBPE_Mistral',
       'GS_CPT_Filtering_AdaptBPE_Mistral', 'GS_CPT_Lookup_Mistral',
       ...
       'Medical_Words_SplitMoreThan2_SD_Llama',
       'Medical_Words_SplitMoreThan3_SD_Llama', 'CSr_Base_Llama',
       'CSr_Llama3.1', 'CSr_CPT_Without_Vocab_Llama', 'CSr_CPT_MedVoc_Llama',
       'CSr_CPT_MedVoc_AdaptBPE_Llama',
       'CSr_CPT_Filtering_WithoutAdaptBPE_Llama',
       'CSr_CPT_Filtering_AdaptBPE_Llama', 'CSr_CPT_Lookup_Llama'],
      dtype='object', length=101)

In [5]:
df_merged['Novel_RS_Llama'].describe(percentiles=[0.9]), df_merged['Novel_RS_Mistral'].describe(percentiles=[0.9])

(count    500.000000
 mean       0.413146
 std        0.112472
 min        0.117647
 50%        0.416667
 90%        0.556000
 max        0.789474
 Name: Novel_RS_Llama, dtype: float64,
 count    500.000000
 mean       0.413146
 std        0.112472
 min        0.117647
 50%        0.416667
 90%        0.556000
 max        0.789474
 Name: Novel_RS_Mistral, dtype: float64)

In [6]:
df_high_oov = df_merged[df_merged['Novel_RS_Llama'] >= 0.556000]
df_desc = pd.DataFrame()
for column in df_high_oov.columns:
     if column.startswith('CSr'): df_desc[column] = df_high_oov[column]
     if column.startswith('R-L'): df_desc[column] = df_high_oov[column]
     if column.startswith('GS'): df_desc[column] = df_high_oov[column]

df_desc['SD'] = df_high_oov['SD']
df_desc['RS'] = df_high_oov['RS_Mistral']
df_desc.describe(percentiles=[0.5])

,R-L_Base_Mistral,R-L_BioMistral,R-L_CPT_Without_Vocab_Mistral,R-L_CPT_MedVoc_Mistral,R-L_CPT_MedVoc_AdaptBPE_Mistral,R-L_CPT_Filtering_WithoutAdaptBPE_Mistral,R-L_CPT_Filtering_AdaptBPE_Mistral,R-L_CPT_Lookup_Mistral,CSr_Base_Mistral,CSr_BioMistral,...,R-L_CPT_Filtering_AdaptBPE_Llama,R-L_CPT_Lookup_Llama,CSr_Base_Llama,CSr_Llama3.1,CSr_CPT_Without_Vocab_Llama,CSr_CPT_MedVoc_Llama,CSr_CPT_MedVoc_AdaptBPE_Llama,CSr_CPT_Filtering_WithoutAdaptBPE_Llama,CSr_CPT_Filtering_AdaptBPE_Llama,CSr_CPT_Lookup_Llama
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,...,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,0.194030,0.214815,0.205900,0.213353,0.203107,0.208154,0.207221,0.217415,0.146134,0.161712,...,0.215913,0.226934,0.156773,0.161621,0.189340,0.172242,0.194621,0.151490,0.186305,0.174490
std,0.082433,0.101331,0.073821,0.096212,0.075268,0.082200,0.081739,0.090784,0.132043,0.122160,...,0.085359,0.088155,0.142565,0.134757,0.165187,0.159536,0.171605,0.147723,0.162821,0.164773
min,0.043478,0.000000,0.072727,0.038462,0.086957,0.066667,0.000000,0.088889,0.000000,0.000000,...,0.068966,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.182105,0.196687,0.188988,0.193548,0.200000,0.205345,0.205263,0.199716,0.108187,0.160000,...,0.216542,0.222222,0.105263,0.155405,0.172161,0.142857,0.143590,0.117647,0.148352,0.143590
max,0.456140,0.529412,0.388060,0.510638,0.480000,0.436364,0.489796,0.510638,0.580645,0.521739,...,0.521739,0.511628,0.625000,0.500000,0.666667,0.666667,0.625000,0.625000,0.666667,0.714286


In [7]:
df_desc = df_desc[df_desc['CSr_Base_Llama']<df_desc['CSr_CPT_Lookup_Llama']]
df_desc = df_desc[df_desc['CSr_Base_Mistral']<df_desc['R-L_CPT_Lookup_Mistral']]

df_desc = df_desc[df_desc['R-L_Llama3.1']<df_desc['R-L_CPT_Lookup_Llama']]
df_desc = df_desc[df_desc['R-L_BioMistral']<df_desc['R-L_CPT_Lookup_Mistral']]


In [8]:
df_desc

,GS_Base_Mistral,GS_BioMistral,GS_CPT_Without_Vocab_Mistral,GS_CPT_MedVoc_Mistral,GS_CPT_MedVoc_AdaptBPE_Mistral,GS_CPT_Filtering_WithoutAdaptBPE_Mistral,GS_CPT_Filtering_AdaptBPE_Mistral,GS_CPT_Lookup_Mistral,R-L_Base_Mistral,R-L_BioMistral,...,CSr_Base_Llama,CSr_Llama3.1,CSr_CPT_Without_Vocab_Llama,CSr_CPT_MedVoc_Llama,CSr_CPT_MedVoc_AdaptBPE_Llama,CSr_CPT_Filtering_WithoutAdaptBPE_Llama,CSr_CPT_Filtering_AdaptBPE_Llama,CSr_CPT_Lookup_Llama,SD,RS
84,The Omega-3 Index is a new risk factor for dea...,The Omega-3 Index is a new risk factor for dea...,The Omega-3 Index is a new risk factor for dea...,The Ogene-3 Index is a new risk factor for dea...,The Oomega-3 Index is a new risk factor for de...,The Omega-3 Index is a new risk factor for dea...,The Omega-3 Index is a new risk factor for dea...,The Omega-3 Index is a new risk factor for dea...,0.372881,0.300000,...,0.240000,0.315789,0.315789,0.315789,0.315789,0.333333,0.285714,0.315789,The Omega-3 Index: a new risk factor for death...,"The Omega-3 Index may represent a novel, physi..."
92,"In this study, we found that viral infections ...",This study suggests that viral infections may ...,The results of this study suggest that viral i...,Viral infections are associated with BPPV. Sea...,Viral infections are more prevalent in patient...,Viral infections are associated with BPPV. Sea...,Viral infections are associated with BPPV. Sea...,Viral infections are associated with BPPV. Sea...,0.184211,0.212121,...,0.000000,0.344828,0.344828,0.000000,0.000000,0.222222,0.080000,0.064516,Do viral infections have a role in benign paro...,BPPV is associated with positive viral serolog...
182,There was no difference in the frequency of nu...,There was no difference in the frequency of nu...,No differences were noted in the frequency of ...,There was no evidence that occupational exposu...,There was no evidence that occupational exposu...,There was no evidence that occupational exposu...,There is no evidence that occupational exposur...,There is no evidence that occupational exposur...,0.149254,0.156250,...,0.000000,0.000000,0.000000,0.111111,0.105263,0.000000,0.100000,0.111111,Does occupational nuclear power plant radiatio...,The data suggest that occupational exposure du...
217,The OraQuick® HIV-1/2 Assay in sputum specimen...,The OraQuick® HIV-1/2 Assay in sputum specimen...,The OraQuick HIV-1/2 Assay in sputum specimens...,The OraQuick HIV-1/2 Assay in sputum is a vali...,The OraQuick HIV-1/2 Assay in sputum is a vali...,The OraQuick HIV-1/2 Assay in sputum is a vali...,The OraQuick HIV-1/2 Assay in sputum is a vali...,The OraQuick HIV-1/2 Assay in sputum is a vali...,0.205128,0.179104,...,0.137931,0.000000,0.121212,0.235294,0.258065,0.148148,0.266667,0.153846,An HIV1/2 point of care test on sputum for scr...,This testing method may provide a useful strat...
343,Achilles tendon structure was similar in T1DM ...,Achilles tendon structure deteriorates 2-days ...,Type 1 diabetes mellitus did not alter Achille...,Achilles tendon structure was similar in T1DM ...,Achilles tendon structure was similar in T1DM ...,The Achilles tendon load-response in individua...,Achilles tendon structure was similar in T1DM ...,There was no evidence of a different load-resp...,0.123077,0.179487,...,0.095238,0.000000,0.090909,0.090909,0.111111,0.117647,0.000000,0.105263,Does type 1 diabetes mellitus affect Achilles ...,Active individuals with T1DM do not have a hei...
428,Variation in utilisation of CEA was considerab...,There was considerable variation in the use of...,Variation in utilisation of CEA is not explain...,Variation in the use of CEA is not explained b...,There is considerable variation in the use of ...,Variation in use of CEA was considerable. Ther...,variation in the use of CEA was not explained ...,variation in utilisation of CEA is not explain...,0.135135,0.190476,...,0.250000,0.272727,0.315789,0.307692,0.333333,0.000000,0.250000,0.333333,Are variations in the use of carotid endartere...,"use of

In [13]:
df_llama.loc[343]

SD                                   Does type 1 diabetes mellitus affect Achilles ...
RS                                   Active individuals with T1DM do not have a hei...
GS_Base                              This study found no evidence of altered Achill...
GS_Llama3.1                          In this study, Achilles tendon structure was q...
GS_CPT_Without_Vocab                 The results suggest that T1DM does not affect ...
GS_CPT_MedVoc                        In this study, Achilles tendon structure was n...
GS_CPT_MedVoc_AdaptBPE               In this study, Achilles tendon load-response w...
GS_CPT_Filtering_WithoutAdaptBPE     In this study, Achilles tendon structure was n...
GS_CPT_Filtering_AdaptBPE            In this study, Achilles tendon structure was n...
GS_CPT_Lookup                        In this study, T1DM did not affect Achilles te...
RS_Len_Base                                                                         52
RS_Len_Filtering_WithoutAdaptBPE           

In [10]:
df_print = df_desc.nlargest(10,['R-L_CPT_Lookup_Mistral','R-L_CPT_Lookup_Llama'])

In [11]:
for idx,row in df_print.iterrows():
    print('-'*200,idx)
    # print('OOVConcentration', row['Medcial_Words_SplitMoreThan3'], row['Medcial_Words_SplitMoreThan3'])
    print('++++++++++++Csr',  row['CSr_Base_Llama'], row['CSr_CPT_Lookup_Llama'], row['CSr_Llama3.1'], row['CSr_Base_Mistral'],row['CSr_CPT_Lookup_Mistral'] , row['CSr_BioMistral'])
    print('++++++++++++R-L',  row['CSr_Base_Llama'], row['R-L_CPT_Lookup_Llama'], row['R-L_Llama3.1'], row['R-L_Base_Mistral'],row['R-L_CPT_Lookup_Mistral'] , row['R-L_BioMistral'])
    print('-- SD: ',row['SD'])
    print('-- RS                    :',row['RS'])
    print('***')
    print('-- GS_Base_Mistral       :',row['GS_Base_Mistral'])
    print('-- GS_Lookup_Mistral     :',row['GS_CPT_Lookup_Mistral'])
    print('-- GS_Base_Llama2        :',row['GS_Base_Llama'])
    print('-- GS_Lookup_Llama2      :',row['GS_CPT_Lookup_Llama'])
    print('-- GS_Llama3.1           :',row['GS_Llama3.1'])    
    print('-- GS BioMistral         :',row['GS_BioMistral'])

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- 84
++++++++++++Csr 0.24 0.3157894736842105 0.3157894736842105 0.3157894736842105 0.3157894736842105 0.2857142857142856
++++++++++++R-L 0.24 0.3999999999999999 0.3225806451612903 0.3728813559322033 0.3999999999999999 0.3
-- SD:  The Omega-3 Index: a new risk factor for death from coronary heart disease?
Low intakes or blood levels of eicosapentaenoic and docosahexaenoic acids (EPA + DHA) are independently associated with increased risk of death from coronary heart disease (CHD). In randomized secondary prevention trials, fish or fish oil have been demonstrated to reduce total and CHD mortality at intakes of about 1 g/day. Red blood cell (RBC) fatty acid (FA) composition reflects long-term intake of EPA + DHA. We propose that the RBC EPA + DHA (hereafter called the Omega-3 Index) be conside

In [ ]:
df_desc.columns